In [2]:
import pandas as pd
import numpy as np
import os

# === CONFIGURATION ===
# Put the folder that contains your 18 CSV files
FOLDER = r"C:\Users\mahdi\Desktop\AI health coach\AI-Health-Coach\FitBit Dataset"   # <--- modify this!

def detect_possible_keys(df):
    # Common identifiers / time keys
    key_candidates = [c for c in df.columns 
                      if "time" in c.lower() 
                      or "date" in c.lower()
                      or "id" in c.lower()
                      or "stamp" in c.lower()]
    return key_candidates

def detect_frequency(df, key):
    try:
        ts = pd.to_datetime(df[key], errors='coerce')
        ts = ts.dropna().sort_values()

        if len(ts) < 2:
            return "Not enough timestamps"

        diffs = ts.diff().dropna()
        most_common = diffs.value_counts().idxmax()

        return str(most_common)
    except:
        return "Could not determine"

print("📌 Scanning folder:", FOLDER)
print("="*70)

for file in os.listdir(FOLDER):
    if file.endswith(".csv"):
        path = os.path.join(FOLDER, file)

        print(f"\n📄 FILE: {file}")
        print("-"*70)

        try:
            # read small preview first (avoid memory crash)
            df = pd.read_csv(path, nrows=5000)

            # read full to get total number of rows
            rows = sum(1 for _ in open(path)) - 1

            print(f"✔️ Total rows: {rows}")
            print(f"✔️ Columns: {list(df.columns)}")

            # detect potential keys
            keys = detect_possible_keys(df)
            print(f"🔑 Possible key columns: {keys}")

            # detect frequency for timestamp-like keys
            for key in keys:
                freq = detect_frequency(df, key)
                print(f"   ⤷ Frequency for {key}: {freq}")

        except Exception as e:
            print("❌ Error reading file:", e)


📌 Scanning folder: C:\Users\mahdi\Desktop\AI health coach\AI-Health-Coach\FitBit Dataset

📄 FILE: dailyActivity_merged.csv
----------------------------------------------------------------------
✔️ Total rows: 940
✔️ Columns: ['Id', 'ActivityDate', 'TotalSteps', 'TotalDistance', 'TrackerDistance', 'LoggedActivitiesDistance', 'VeryActiveDistance', 'ModeratelyActiveDistance', 'LightActiveDistance', 'SedentaryActiveDistance', 'VeryActiveMinutes', 'FairlyActiveMinutes', 'LightlyActiveMinutes', 'SedentaryMinutes', 'Calories']
🔑 Possible key columns: ['Id', 'ActivityDate']
   ⤷ Frequency for Id: 0 days 00:00:00
   ⤷ Frequency for ActivityDate: 0 days 00:00:00

📄 FILE: dailyCalories_merged.csv
----------------------------------------------------------------------
✔️ Total rows: 940
✔️ Columns: ['Id', 'ActivityDay', 'Calories']
🔑 Possible key columns: ['Id']
   ⤷ Frequency for Id: 0 days 00:00:00

📄 FILE: dailyIntensities_merged.csv
--------------------------------------------------------------

C:\Users\mahdi\AppData\Local\Temp\ipykernel_18260\3405319067.py:20: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  ts = pd.to_datetime(df[key], errors='coerce')


✔️ Total rows: 17141606
✔️ Columns: ['Id', 'ActivityDate', 'TotalSteps', 'TotalDistance', 'TrackerDistance', 'LoggedActivitiesDistance', 'VeryActiveDistance_x', 'ModeratelyActiveDistance_x', 'LightActiveDistance_x', 'SedentaryActiveDistance_x', 'VeryActiveMinutes_x', 'FairlyActiveMinutes_x', 'LightlyActiveMinutes_x', 'SedentaryMinutes_x', 'Calories_x', 'ActivityDay_x', 'Calories_y', 'ActivityDay_y', 'SedentaryMinutes_y', 'LightlyActiveMinutes_y', 'FairlyActiveMinutes_y', 'VeryActiveMinutes_y', 'SedentaryActiveDistance_y', 'LightActiveDistance_y', 'ModeratelyActiveDistance_y', 'VeryActiveDistance_y', 'ActivityDay', 'StepTotal']
🔑 Possible key columns: ['Id', 'ActivityDate']
   ⤷ Frequency for Id: 0 days 00:00:00
   ⤷ Frequency for ActivityDate: 0 days 00:00:00


In [3]:
def parse_dates(df):
    for col in df.columns:
        if "date" in col.lower() or "time" in col.lower():
            df[col] = pd.to_datetime(df[col], errors="coerce")
    return df


In [6]:
import pandas as pd
import os

# === Set your folder path ===
DATA_DIR = r"C:\Users\mahdi\Desktop\AI health coach\AI-Health-Coach\FitBit Dataset"

def parse_dates(df):
    for col in df.columns:
        if any(x in col.lower() for x in ["date", "time", "hour", "minute"]):
            df[col] = pd.to_datetime(df[col], errors="coerce")
    return df

def safe_merge(df1, df2, keys):
    overlap = set(df1.columns) & set(df2.columns) - set(keys)
    df2 = df2.rename(columns={c: f"{c}_y" for c in overlap})
    df1 = df1.rename(columns={c: f"{c}_x" for c in overlap})
    return pd.merge(df1, df2, on=keys, how="outer")


# === DAILY FILES ===
daily_df = None

for file in daily_files:
    df = pd.read_csv(file)

    # Standardize column names
    df = df.rename(columns={
        "ActivityDay": "ActivityDate",
        "SleepDay":   "ActivityDate",
        "Date":       "ActivityDate"
    })

    # FORCE ActivityDate → datetime for ALL files
    if "ActivityDate" in df.columns:
        df["ActivityDate"] = pd.to_datetime(df["ActivityDate"], errors="coerce")

    # parse other time/date columns
    df = parse_dates(df)

    if daily_df is None:
        daily_df = df
    else:
        daily_df = safe_merge(daily_df, df, keys=["Id", "ActivityDate"])



# === HOURLY FILES ===
hourly_files = [
    os.path.join(DATA_DIR, "hourlyCalories_merged.csv"),
    os.path.join(DATA_DIR, "hourlyIntensities_merged.csv"),
    os.path.join(DATA_DIR, "hourlySteps_merged.csv"),
]

hourly_df = None
for file in hourly_files:
    df = pd.read_csv(file)
    df = parse_dates(df)
    df = df.rename(columns={"ActivityHour": "DateTime"})

    if hourly_df is None:
        hourly_df = df
    else:
        hourly_df = safe_merge(hourly_df, df, keys=["Id", "DateTime"])


# === MINUTE FILES (narrow) ===
minute_files = [
    os.path.join(DATA_DIR, "minuteCaloriesNarrow_merged.csv"),
    os.path.join(DATA_DIR, "minuteIntensitiesNarrow_merged.csv"),
    os.path.join(DATA_DIR, "minuteStepsNarrow_merged.csv"),
    os.path.join(DATA_DIR, "minuteMETsNarrow_merged.csv"),
]

minute_df = None
for file in minute_files:
    df = pd.read_csv(file)
    df = parse_dates(df)
    df = df.rename(columns={"ActivityMinute": "DateTime"})

    if minute_df is None:
        minute_df = df
    else:
        minute_df = safe_merge(minute_df, df, keys=["Id", "DateTime"])


# === SECOND FILE ===
seconds_df = pd.read_csv(os.path.join(DATA_DIR, "heartrate_seconds_merged.csv"))
seconds_df = parse_dates(seconds_df)
seconds_df = seconds_df.rename(columns={"Time": "DateTime"})

# === SAVE OUTPUTS ===
daily_df.to_csv(os.path.join(DATA_DIR, "merged_daily.csv"), index=False)
hourly_df.to_csv(os.path.join(DATA_DIR, "merged_hourly.csv"), index=False)
minute_df.to_csv(os.path.join(DATA_DIR, "merged_minute.csv"), index=False)
seconds_df.to_csv(os.path.join(DATA_DIR, "merged_seconds.csv"), index=False)

print("Done!")


C:\Users\mahdi\AppData\Local\Temp\ipykernel_18260\1075029515.py:35: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["ActivityDate"] = pd.to_datetime(df["ActivityDate"], errors="coerce")
C:\Users\mahdi\AppData\Local\Temp\ipykernel_18260\1075029515.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors="coerce")
C:\Users\mahdi\AppData\Local\Temp\ipykernel_18260\1075029515.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors="coerce")
C:\Users\mahdi\AppData\Local\Temp\ipykernel_18260\1075029515.py

Done!
